# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes



### Import and Configuration

In [1687]:
import sympy
import random
import coloredlogs, logging
coloredlogs.install(fmt='%(levelname)s %(message)s')

In [1665]:
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [1666]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10



msg = 12345678900987654321

In [1688]:
def genPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def genSlightlySmallerPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def genPrimeWithLimit(lb, ub):
    return sympy.randprime(lb, ub)

def genSmallPrime():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def genRandomNumberWithBitLimit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermatPrime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def mmi(E, phiN):
    return pow(E, -1, phiN)

def calcN(P, Q):
    return P*Q

def calcPhiN(P, Q):
    return (P-1) * (Q-1)

def toBinary(dec):
    return "{0:b}".format(dec)

def toDecimal(bin):
    return int(bin, 2)

def concatenateInBinary(prev, tail):
    prev = toBinary(prev)
    tail = toBinary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return toDecimal(prev + tail)

def splitInBinary(bin):
    bin = toBinary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(toDecimal(prev), toDecimal(tail))



# Voraussetzungen

In [1668]:
def createAttackerKey():
    a_P = genSmallPrime()
    a_Q= genSmallPrime()


    a_PhiN = calcPhiN(a_P, a_Q)
    a_N = calcN(a_P, a_Q)

    a_E = fermatPrime()
    a_D = mmi(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

Attacker = {}

created = False
while not created:
    try:
        Attacker = createAttackerKey()
        created = True
    except ValueError:
            print("Trying to generate Key failed. Trying again ...")

print(Attacker)

{'P': 2873176535591725830538869002308395370556685101753083363895598418028835063061424987963005472654207868956543402084964541806400874787710058696332860441987109, 'Q': 6953551524306915817853871351382159506450316347893192739591658895046602305963328867297575884795986644964783944450344078219509394059948384733381732136098647, 'PhiN': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566833028792614673074573887023340940110217554493658083333368967291438667619003437840111888498520783222344894651259273431376294848336542043690258172462948255768, 'N': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566842855520674571716222279763694630665094561495107729609472454548751743056372462593967149079878233416858815978605808739996320758605389702133687887055526341523, 'E': 17, 'D': 152778914130980713718184865857

# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [1669]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
User = {}

valid = False
while not valid:
    # Schritt 1
    P = genSlightlySmallerPrime()
    vP = pow(P, Attacker["E"] ,Attacker["N"])

    # Schritt 2
    t = genRandNumber() 
    tempN = concatenateInBinary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(genPrimeWithLimit(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    User["P"] = P
    User["Q"] = Q

Attacker["vP"] = vP
Attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

print(User["P"])
x = pow(User["P"], Attacker["E"], Attacker["N"])
# print(x)

y = pow(x, Attacker["D"], Attacker["N"])

print(y)



INFO Bounds: [1537075691119557241760313157898263413970120767000946119881189859511460583484904173927450555920076672662423223036913158525515786183271997585377693676753330083287719735117946763478832859121594838350075915359295381671802268196350779075667034128725562492761522229028635495985337382752234145834122966257574560430574625446, 1537075691119557241760313157898263413970120767000946119881189859511460583484904173927450555920076672662423223036913158525515786183271997585377693676753330083287719735117946763478832859121594838350075915359295381671802268196350779075667034128725562492761522229028635495985337382752234145834122966257574560430653291223] 	 P: 2285228984691362359507248701760439716194724544893935207134173463810164796933502871014407808351759062446489872060409412727337618943974286670414211660420749843047808840552506032522349730225464427901691129344452652114088730425114808177585670388116631710953742555791933919640245411293608547697907533482197 	 deltaB: 78665777
INFO Amount of failed at

2285228984691362359507248701760439716194724544893935207134173463810164796933502871014407808351759062446489872060409412727337618943974286670414211660420749843047808840552506032522349730225464427901691129344452652114088730425114808177585670388116631710953742555791933919640245411293608547697907533482197
2285228984691362359507248701760439716194724544893935207134173463810164796933502871014407808351759062446489872060409412727337618943974286670414211660420749843047808840552506032522349730225464427901691129344452652114088730425114808177585670388116631710953742555791933919640245411293608547697907533482197


## Schritt 4

In [1670]:
User["N"] = calcN(User["P"], User["Q"])

## Schritt 5

In [1671]:
def createUserKey():
    PhiN = calcPhiN(P, Q)
    E = fermatPrime()
    D = mmi(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        User.update(createUserKey())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [1672]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [1673]:
pub_User = {
    "N" : User["N"],
    "E" : User["E"]
}

collection_User = {
    "N" : pub_User["N"],
    "E" : pub_User["E"]
}

## Schritt 2

In [1674]:
collection_User["vP"] = splitInBinary(User["N"])[0]
collection_User["vP'"] = collection_User["vP"]+1

### Validation test for vP

In [1675]:
testVar = "vP"
altTestVar = "vP'"

if(collection_User[testVar] == Attacker[testVar] or collection_User[altTestVar] == Attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, Attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(Attacker[testVar], collection_User[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 19539318768550257780758417149877080321382872490787390478869188819985280433486595344647033363208397816089543584495066904464245075176466460156243147173629275890099778068571231768575509644980238097785060162631929765301767318442436247969928619587327572004743240690710361029320002431120689968392382166310025282796!
INFO Current vP: 19539318768550257780758417149877080321382872490787390478869188819985280433486595344647033363208397816089543584495066904464245075176466460156243147173629275890099778068571231768575509644980238097785060162631929765301767318442436247969928619587327572004743240690710361029320002431120689968392382166310025282795 	 Current vP': 195393187685502577807584171498770803213828724907873904788691888199852804334865953446470333632083978160895435844950669044642450751764664601562431471736292758900997780685712317685755096449802380977850601626319297653017673184424362479699286195873275720047432406907103610293200024311

## Schritt 3

In [1676]:
collection_User["P"] = pow(collection_User["vP"], Attacker["D"], Attacker["N"])
collection_User["P'"] = pow(collection_User["vP'"], Attacker["D"], Attacker["N"])

print(Attacker)
print(User)
print(collection_User)


{'P': 2873176535591725830538869002308395370556685101753083363895598418028835063061424987963005472654207868956543402084964541806400874787710058696332860441987109, 'Q': 6953551524306915817853871351382159506450316347893192739591658895046602305963328867297575884795986644964783944450344078219509394059948384733381732136098647, 'PhiN': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566833028792614673074573887023340940110217554493658083333368967291438667619003437840111888498520783222344894651259273431376294848336542043690258172462948255768, 'N': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566842855520674571716222279763694630665094561495107729609472454548751743056372462593967149079878233416858815978605808739996320758605389702133687887055526341523, 'E': 17, 'D': 152778914130980713718184865857

### Validation test for P and P'

In [1677]:
testVar = "P"
altTestVar = "P'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}".format(testVar, altTestVar, User["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(User[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables P or P' are in the expected state: 2285228984691362359507248701760439716194724544893935207134173463810164796933502871014407808351759062446489872060409412727337618943974286670414211660420749843047808840552506032522349730225464427901691129344452652114088730425114808177585670388116631710953742555791933919640245411293608547697907533482197!
INFO Current P: 3426542763917660755200058287474563341624102111560605122433520412433064919478012682925459111074317184820999601413181945373111656092256999438022416191145435976522266209561139161048869706999002266413920206425924563670917655475715631647802139211092031393432762604456500829662953601574420131893849808162393900724 	 Current P': 2285228984691362359507248701760439716194724544893935207134173463810164796933502871014407808351759062446489872060409412727337618943974286670414211660420749843047808840552506032522349730225464427901691129344452652114088730425114808177585670388116631710953742555791933919640245411293608547697

# Schritt 4

In [1678]:
# Potential to optimize when N/P not hole Number
collection_User["Q"] = int(pub_User["N"]//collection_User["P"])
collection_User["Q'"] = int(pub_User["N"]//collection_User["P'"])

### Validation test for Q and Q'

In [1679]:
testVar = "Q"
altTestVar = "Q'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables Q or Q' are in the expected state!
INFO Current Q: 1025106109282845173196726779325443631936488949774059897399967498949876571805419078886640012293105328048597510304047344807242379789556012483667247747112470592740073912517133602085669286894375145552429659488381958433982085845388163021904073143539919329590749657024105679887049736067979450324611703548518760818563 	 Current Q': 1537075691119557241760313157898263413970120767000946119881189859511460583484904173927450555920076672662423223036913158525515786183271997585377693676753330083287719735117946763478832859121594838350075915359295381671802268196350779075667034128725562492761522229028635495985337382752234145834122966257574560430581085603


## Schritt 5

In [1680]:
collection_User["PhiN"] = int(calcPhiN(collection_User["P"], collection_User["Q"]))
collection_User["PhiN'"] = int(calcPhiN(collection_User["P'"], collection_User["Q'"]))

collection_User["D"] = mmi(pub_User["E"], collection_User["PhiN"])
collection_User["D'"] = mmi(pub_User["E"], collection_User["PhiN'"])

### Validation test for PhiN and PhiN'

In [1681]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables PhiN or PhiN' are in the expected state!
INFO Current PhiN: 3512569921010919894671414903872549543197421360704493672024423738568624566365434084214942216526946425623482118453806389669107027509638172853470062549091240334245919103370408587648480967864672715187771030848909340595762870257521445174550411097788389579864091274571210030575527923705734924682457589068433495678415159733761412385260741765281514549122089963779599094963293302346036697972597125297407751456546741499142301729052661491052910148128249864953687041913443785714196773090556705189761914593800298630247358519948048659667382322937447588414925519012023507615032914432788878081213113733567423273628994168539643620326 	 Current PhiN': 3512569921010919894671414903872549543197421360704493672024423738568624566365434084214942216526946425623482118453806389669107027509638172853470062549091240334245919103370408587648480967864672715187771030848909340595762870257521445174550411097788389579864091274571210030575

### Validation test for D and D'

In [1682]:
testVar = "D"
altTestVar = "D'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables D or D' are in the expected state!
INFO Current D: 751717298270819432711781399661440563719292509100183470666705469343479965564587060824209423770358184471951426128246503625684383319183266563972192374319137036511772570760204172453955070943801553833958780920972816080805283518146612780545807822483896602694649883663099422885813368886441326293911157193633627479816473872983959848985761856383203502730408358007307199311210628906739371161450746658978312568521676196314500370030725221820661704852349192888921351382254506670353395019379839632050215185443643675733870500377987067243992325920465437209419858154324096960415604256044312429831600215354896031321380074979301163883 	 Current D': 21731463713647325418395135008394372660248637990350758515637485386475148095412607758372599705361263878370959409889308013906148535954570797031196106821225961600976697954704084258214337505466263101745353848442668683063280014433692987655778807959079919968808969364078692407062604664171667

## Schritt 6

In [1683]:
collection_User["Sig"] = pow(msg, collection_User["D"], User["N"])
collection_User["Sig'"] = pow(msg, collection_User["D'"], User["N"])

### Validation test for signature and signature'

In [1684]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_User[testVar] == signature):
    collection_User["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_User[testVar]))
elif(collection_User[altTestVar] == signature):
    collection_User["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_User[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D' is the right privat Key: 3297104653861369492756370481108096459990812036294693431491950256233241481626435899648143214248483576143528327696026858829815067960542631708144775330275838910463073969093672196493796819435951490874001519482416157509610284287477682573592132475518939399038221822575807364414660070134971450698287040055527743729927219422192279811948390764286543274682499223336745449164265000111742817283829525552237760416757983362044705585746628818252858005188745256349945330593838723986441409210361367963301336776754372175174326500748213601318853373516083917741118678254497401346032306431389677335986263858384926698323813972223903939385


# Result

In [1685]:
def sortDic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [1686]:
print(sortDic(Attacker))
print(sortDic(User))
print(sortDic(collection_User))



{'D': 15277891413098071371818486585712852470650711956136955014237450787565998487259435251818073705728686459735452199282332330049701826837598725713648750331551107610253175926468791795959025424790166365201032651960811563222864863473355570113026738263574716581793154733315914976934813707551473327527844484824607489705, 'E': 17, 'N': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566842855520674571716222279763694630665094561495107729609472454548751743056372462593967149079878233416858815978605808739996320758605389702133687887055526341523, 'P': 2873176535591725830538869002308395370556685101753083363895598418028835063061424987963005472654207868956543402084964541806400874787710058696332860441987109, 'PhiN': 19978781078666708716993405535162960923158623327256018095541281799124767252570030713915942538260589985807899029830742277757302388941475256702463750433566833028792614673074573887023340940110217